
# 使用 Ollama 的本地大语言模型介绍

是时候动手实践了！

本笔记本演示如何使用 Ollama 和 LangChain 运行本地大语言模型 (LLM)。这将完全在您的计算机上运行。

Ollama 允许我们在本地机器上运行 LLM。LangChain 是一个 SDK（库，即可重用代码），使与 LLM 的交互变得简单。

## 1. 先决条件

在运行此笔记本之前，您需要安装 Ollama 并下载一个模型。请按照以下步骤操作。

### 1.1 安装 Ollama

**macOS:**
- 从 [ollama.ai](https://ollama.ai) 下载安装程序
- 打开下载的文件并按照安装提示操作

**Windows:**
- 从 [ollama.ai](https://ollama.ai) 下载安装程序
- 运行安装程序并按照说明操作

**Linux:**

In [ ]:
%%bash
curl -fsSL https://ollama.ai/install.sh | sh

验证您的安装（Windows、Mac 和 Linux）：

In [ ]:
%%bash
ollama --version

### 1.2 下载 TinyLlama 模型

下载 TinyLlama 模型（约 600MB）。Ollama 有一个很大的模型目录，您可以从中下载包括 DeepSeek 和其他模型，但 TinyLlama 是最小的，非常适合演示目的：

In [ ]:
%%bash
ollama pull tinyllama:1.1b

## 2. 设置 Python 环境

安装所需的包。我们将安装 Langchain，这是与 LLM 交互最流行的库之一。Langchain-ollama 允许我们与本地模型交互。

In [ ]:
%%bash
pip install langchain langchain-ollama

## 3. 使用 LangChain 与 LLM 交互

下面的代码允许我们与本地计算机上托管的 tinyllama 模型交互。我们只是向模型发送以下消息：
。

如果您想确切了解发生了什么，请阅读代码注释。

In [ ]:
# Import the libraries
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage

# Initialize the LLM. This is when we pick the model and the temperature (which controls the randomness of the output). llm is what we will use to call the model.
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0,  # 0 for more deterministic outputs
)

# We need to provide an array of messages to the model. The first message is always the system message, which tells the model what it is. The second message is the human message, which is what we want to ask the model.
messages = [
    (
        "system",
        "You are a helpful assistant",
    ),
    ("human", "I love programming."),
]

# Send the array of messages to the model and get the response. The response is an AIMessage object, which contains the content of the message. Which we will print below.
ai_msg = llm.invoke(messages)

# Display the response
print(ai_msg.content)

## 4. 测试不同的提示

让我们尝试几个例子，通过更改消息数组并使用不同的消息来看看模型能做什么：

In [ ]:
# Ask for an explanation
messages = [
    (
        "system",
        "You are a helpful and informative AI assistant.",
    ),
    ("human", "Explain the concept of a neural network in simple terms."),
]

ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
# Ask for some code
messages = [
    (
        "system",
        "You are an expert Python programmer.",
    ),
    ("human", "Write a function to check if a string is a palindrome."),
]

ai_msg = llm.invoke(messages)
print(ai_msg.content)

## 5. 创建基本聊天机器人

让我们创建一个简单的聊天界面，这样我们就可以与模型交互。每个单元格将代表一次交互。按照以下步骤操作，当您完成这些步骤时会更有意义。

In [ ]:
# Setup: Run this cell first
from langchain_ollama import ChatOllama

# Initialize the LLM
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0
)

# System prompt
system_prompt = "You are a helpful assistant"

print("-" * 30)

现在让我们创建一个单元格来进行第一次交互，告诉模型我们最喜欢的颜色是什么。请注意，响应可能会很奇怪，我们没有使用最好的模型。但它应该表明它理解了您最喜欢的颜色是蓝色。

In [ ]:
# First interaction 
user_message = "My favorite color is blue."

# Create fresh messages for this interaction only
messages = [
    ("system", system_prompt),
    ("human", user_message)
]

# Get response
ai_msg = llm.invoke(messages)

print(f"You: {user_message}")
print(f"Assistant: {ai_msg.content}")

现在让我们跟进一个问题：

In [ ]:
# Second interaction
user_message = "What's my favorite color?"

# Create fresh messages for this interaction only
messages = [
    ("system", system_prompt),
    ("human", user_message)
]

# Get response
ai_msg = llm.invoke(messages)

print(f"You: {user_message}")
print(f"Assistant: {ai_msg.content}")

如您所见，模型不记得我们最喜欢的颜色！您可能会问为什么？ChatGPT 和其他工具当然可以记住对话的完整上下文。这就是记忆的概念。LLM 本身不记得过去的对话。我们需要设计我们的应用程序使其能够记住。

让我们在下面探索如何做到这一点，以及像 ChatGPT 这样的工具如何管理记忆。


## 6. 理解对话记忆的工作原理

最简单的管理方法是将整个对话历史传递给模型，而不仅仅是单个消息：

In [ ]:
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

# Initialize the LLM like before
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0
)

# Define the conversation history (always starts with a system prompt)
conversation = [SystemMessage(content="You are a helpful assistant")] #conversation is an array (list) of messages. SystemMessage is a class that represents a system message and accepts content as a parameter.

#Define a function called chat which takes user input as a parameter. This function will be used to interact with the model.
def chat(user_input: str):
    # 1) Append the user's message
    conversation.append(HumanMessage(content=user_input)) #We add the user message to the conversation list. Now we have the system message and the user message in the conversation list.
    
    # 2) Call the model with the full conversation list (user message + system message)
    ai_msg = llm(conversation)
    
    # 3) Display the exchange
    print(f"You: {user_input}")
    print(f"Assistant: {ai_msg.content}")
    print(f"[Conversation length: {len(conversation) + 1} messages]")  # +1 for the AI response
    
    # 4) Add the AI's response to history. So now we have the system message, user message, and AI response in the conversation list which will be used for the next interaction.
    conversation.append(ai_msg)
    
    return ai_msg.content

print("Chat With Memory")
print("-" * 30)


让我们再次提供我们最喜欢的颜色：

In [ ]:
# First interaction - share favorite color
chat("My favorite color is blue")

现在让我们看看它是否记住了：

In [ ]:
# Second interaction - test memory
chat("What's my favorite color?")

现在模型应该正确记住您的名字！这是因为我们在跟踪对话历史。

## 7. 上下文窗口挑战

虽然我们简单的记忆解决方案最初是有效的，因为我们只是将所有消息历史作为输入传递，但它有一个关键限制：**上下文窗口**。

### 理解上下文窗口

每个语言模型都有一个固定的
 - 它一次可以处理的最大令牌数（大致是单词或单词片段）：

- **TinyLlama**: ~2,048 个令牌（约 1,500 个单词）
- **GPT-3.5**: ~4,096 个令牌
- **GPT-4**: ~8,192 到 128,000 个令牌，取决于版本
- **Claude 3**: 高达 200,000 个令牌

随着对话的增长，我们最终会达到这个限制。当这种情况发生时：

1. 模型无法看到超出窗口大小的消息
2. 它有效地
对话的最早部分
3. 较大的上下文处理变慢
4. 如果您超出上下文窗口，可能会收到错误

## 8. 上下文窗口问题的解决方案

在像 ChatGPT 这样的生产系统中，有几种技术可以解决上下文窗口限制：

### 1. 滑动上下文窗口
这种方法只保留最近的 N 条消息，加上系统提示。

**示例：** 如果您限制为 10 条消息，而您的对话达到 15 条消息，您会丢弃 5 条最旧的消息（除了系统提示）。

这就像进行对话，您只记得最后几分钟说的话。它很简单但会丢失所有较旧的信息。

### 2. 摘要
这种技术将对话的较旧部分压缩成摘要以节省令牌空间。

**示例：** 在 10 次来回交流后，系统可能会用一个摘要替换这 20 条消息：


这保留了关键点，同时显著减少令牌使用。

### 3. 基于数据库的会话管理

这种方法使用数据库来存储与唯一用户会话关联的完整对话历史。工作原理如下：

1. **会话创建**：当用户开始聊天时，系统创建一个唯一的会话 ID（如
）

2. **消息存储**：用户和 AI 的每条消息都存储在链接到该会话 ID 的数据库表中

3. **上下文窗口管理**：在为 LLM 准备提示时，系统：
   - 检索会话的所有消息
   - 应用策略以适应上下文窗口（如滑动窗口或摘要）
   - 将优化的对话发送给模型
   - 将新响应存储回数据库

**实践中的示例：**
- 用户与 ChatGPT 聊天了几个小时
- 他们的会话 ID 
 现在在数据库中有 200 条消息
- 当他们发送第 201 条消息时，系统：
  - 从数据库检索所有 200 条先前消息
  - 选择最重要的消息以适应上下文窗口
  - 从模型获得响应
  - 将第 201 条消息和响应添加到数据库
  - 即使模型只
最近的部分，完整历史仍保留在数据库中

这就是像 ChatGPT 这样的服务如何能够在非常长的对话中保持
，甚至当您关闭浏览器并稍后返回时也是如此。

## ChatGPT 通常使用什么

ChatGPT 使用结合几种技术的混合方法：

1. **主要技术：** 它使用非常大的上下文窗口（GPT-4o 中高达 128K 令牌）和基于数据库的会话管理，允许它
非常长的对话。

2. **对于长对话：** 当对话超出这些慷慨的限制时，它使用滑动窗口技术，优先考虑：
   - 系统提示/指令
   - 最近的消息
   - 信息密度高的消息
   - 用户明确引用的消息

3. **动态压缩：** 在某些版本中，ChatGPT 还使用动态压缩算法，有选择地摘要或删除对话中似乎与当前交流不太相关的部分。

虽然这些解决方案有所帮助，但任何模型在单次对话中能够
的内容仍有绝对限制。这就是为什么即使是 ChatGPT 有时也会
在非常长的对话中较早提到的事情。

## 9. 本地 LLM 的好处

使用像 Ollama 这样的工具运行本地 LLM 有几个优势：

1. **隐私**: 您的数据永远不会离开您的计算机
2. **无 API 成本**: 随心所欲地运行模型，无需按查询付费
3. **离线使用**: 下载模型后无需互联网连接
4. **无速率限制**: 根据您的硬件能力运行尽可能多的查询

## 10. 进一步学习资源

- [Ollama 文档](https://github.com/ollama/ollama/blob/main/README.md)
- [LangChain 文档](https://python.langchain.com/docs/get_started/introduction)
- [TinyLlama 模型信息](https://github.com/jzhang38/TinyLlama)